<a href="https://colab.research.google.com/github/isharaWijayaratne/TimeSeriesModelSelectionChatbots/blob/main/FFORMS_LLM_monthly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai pinecone-client[grpc] datasets tiktoken langchain pandas gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import openai

#def get_embedding(text, model="text-embedding-ada-002"): # model = "deployment_name"
#    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
#from openai.embeddings_utils import get_embedding

from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.agents import Tool
from langchain.agents import initialize_agent

from tqdm.auto import tqdm # provides a progress bar
from uuid import uuid4 #UUID (Universally Unique Identifier)

import pinecone
import gradio as gr

In [ ]:
df = pd.read_csv('TRAINING_monthly_1000_FINAL.csv')

In [ ]:
!pip install langchain_openai

In [ ]:

from langchain_openai import OpenAIEmbeddings

openai_api_key = 'OPENAI-API KEY'
model_name = 'text-embedding-ada-002'
embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=openai_api_key,
)

In [ ]:
!pip install pinecone-client

from pinecone import Pinecone

pc = Pinecone(api_key="API KEY")
index = pc.Index("businesschatbot", dimension=1536, metric='cosine')
index_name = "businesschatbot"


In [ ]:


#NEED ONLY TO RUN ONCE
batch_size = 5

texts = []
metadatas = []

for i in tqdm(range(0, len(df), batch_size)):
    i_end = min(len(df), i+batch_size)
    batch = df.iloc[i:i_end]

    metadatas = [{
        'text': str(record['Features']),
        'text': str(record['classlabels'])
    } for j, record in batch.iterrows()]
    documents = batch['classlabels']
    batch['classlabels'] = batch['classlabels'].fillna('')
    documents = batch['classlabels'].astype(str)
    embeds = embed.embed_documents(documents)
    ids = batch['Feature_ID'].astype(str)
    index.upsert(vectors=zip(ids, embeds, metadatas))

  0%|          | 0/3300 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['classlabels'] = batch['classlabels'].fillna('')
<ipython-input-9-e31c2fda82fb>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['classlabels'] = batch['classlabels'].fillna('')
<ipython-input-9-e31c2fda82fb>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['classlabels'] = batch['classlabels'].fill

In [ ]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.165,
 'namespaces': {'': {'vector_count': 16500}},
 'total_vector_count': 16500}

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
!pip install --upgrade --quiet  langchain-pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 4.3 MB/s eta 0:00:00


In [ ]:
from langchain_pinecone import PineconeVectorStore

In [ ]:
text_field = "text"

vectorstore = PineconeVectorStore(
    index, embed, text_field
)

In [ ]:
from sqlalchemy.sql.expression import true
llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)


In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)


In [ ]:

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'based on the most similar row of the classlabels column'
            'answer with one of more information'

        )
    )
]

In [ ]:
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=False,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
with gr.Blocks(theme=gr.themes.Soft()) as demo: #"gradio/seafoam"
  # Adding a title using Markdown
  gr.Markdown("# FFORMS Monthly GPT Chatbot")

  chatbot = gr.Chatbot()
  msg = gr.components.Textbox(label="timeseries_features")
  clear = gr.ClearButton([msg, chatbot])

  def respond(message, chat_history):
    bot_message = agent.run(message)
    print(bot_message)
    chat_history.append((message, bot_message))
    return "", chat_history

  msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(share=True, debug=true)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ec0c0ac643a06433f8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


For a time series with a strong consistent seasonal pattern, the best suited forecasting model is the Seasonal ARIMA (SARIMA) model or the Seasonal Decomposition of Time Series (STL) model.
For a time series with strong trend and seasonality, the best suited forecasting model is the Seasonal ARIMA (SARIMA) model.
Seasonal ARIMA (SARIMA) model is a time series forecasting model that extends the ARIMA model to account for seasonality in the data. It includes parameters for seasonal components such as seasonal autoregressive terms, seasonal differences, and seasonal moving average terms. SARIMA models are useful for forecasting time series data that exhibit seasonal patterns.
For a time series with a strong trend and no seasonality, the best suited forecasting model is the ARIMA (Autoregressive Integrated Moving Average) model.
For a complex non-linear time series, the best suited forecasting model is the Long Short-Term Memory (LSTM) neural network model.
Based on the context provided, a